# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

In [3]:
## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. 

In [4]:
# aktywność: napisać test pierwszości oparty tylko o małe twierdzenie fermata (sprawdzamy dla różnyc podstaw)

###### Test pierwszości Rabina-Millera
import random

def rabinMiller(num):
    d = num - 1    ##obliczamy wartości d i sa
    s = 0
    while d % 2 == 0:   ##usuwamy z num-1 dzielniki 2 zliczając je w s  
        d = d // 2
        s += 1
    #finalnie mamy num-1 = 2^s*d
    for trials in range(6):   ## wykonujemy n testów R-B 
        x = random.randrange(2, num - 1)  ##losujemy baze x
        a = pow(x, d, num)   ### pierwszy wyraz ciągu
        if (a != 1) and (a != num-1): ## jeśli a nie spełnia warunków losujemy innego świadka
            i = 1
            while (i < s) and (a != (num - 1)): #generujemy ciąg R-M
                a = (a ** 2) % num   ## obliczamy kolejne wyrazy ciągu R-M: b^d, b^2d, b^4d
                if(a==1):         
                    return False ##To nie jest liczba pierwsza lub silnie pseudopierwsza
                i+=1
            if(a!=num-1):    
                return False  ##To nie jest liczba pierwsza lub silnie pseudopierwsza                
    ### jeśli wykonaliśmy n testów i żaden nie zakończył się False 
    return True

def isPrime(num):
    if (num <= 2):
        return False # oczywista oczywistość 
    #opcjonalne można sprawdzić czy małe liczby pierwsze nie są czynnikami badanej liczby   
    return rabinMiller(num)


isPrime(29)

True

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [38]:
#napisz funkcję generującą liczbę pierwszą 

import random

def generatePrime(keysize, try_limit = 1000, exp_gap = 2):
    assert keysize >= 2 ** exp_gap
    lower_exp_bound, higher_exp_bound = pow(2, keysize - exp_gap), pow(2, keysize)
    num = random.randint(lower_exp_bound, higher_exp_bound)
    try_count = 0
    while not isPrime(num) and try_count < try_limit:
        num = random.randint(lower_exp_bound, higher_exp_bound)
        try_count += 1
        
    if try_count >= try_limit: 
        raise RuntimeError("Failed to find a prime. Try limit exceeded.")
    return num

print(generatePrime(1024))

130381342876699218043876267711260700283927767531500373265291429617920707346794444867915884184779519804114610068298320716234541403346815327084481775742959047267025436224524192750897071922584105862705337956509654042094702390901067161641212169968024439911190167199470065067816892391703945911347390251406477213317


## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [39]:
! pwd

/Users/kacperkafara/studies/cs/5_term/krypto/lab/lab05


In [48]:
import random, sys, os


def generateKey(key_size):
    #napisz swój kod tutaj
    # wybieramy dwie duże liczby pierwsze p i q
    p, q = generatePrime(key_size, try_limit=5000), generatePrime(key_size, try_limit=5000)
    
    # moduł n
    n = p * q
    
    # poszukujemny wukładnika publicznego względnie pierwszego z (p-1) * (q-1)
    # tego nie wiem
    # 65537
    #e = findModInverse(p, (p-1) * (q-1))
    e = 65537
    d = findModInverse(e, (p-1) * (q-1))
    
    
    return ((n, e), (n, d))

def makeKeyFiles(key_size, key_dir = 'keys'):
    public, private = generateKey(key_size)
    with open(f'{key_dir}/public', 'w') as file:
        file.write(str(public[0]) + '\n')
        file.write(str(public[1]) + '\n')
    with open(f'{key_dir}/private', 'w') as file:
        file.write(str(private[0]) + '\n')
        file.write(str(private[1]) + '\n')
  
print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(16)

public_key, private_key = generateKey(2048)
print(public_key)
print(private_key)

Generujemy klucze publiczny i prywatny
(454078622221324603146947209551390321399310773457598909527321536488325207390796542547262105496433508662869189000567864965530346309821970678914607706965519890821165890700762232853340019496769781018386559740214861683833730755019776197383860124559160052225010312198379342429071543239356204960311291135756008401945026372972675679282947877009175763089742443533993861056854223639545964463208951883739311525330639596264717288372493684316669821664269824691110340847280836797124130697977244482727231857690578791883901811603087028408626475328679933785663233457654182853355140321162233119771895184727559513689657547186937332642699567536736192528247096768137144269706416310435034360415353901482838345178389789743855819059277413910629856355032556524001723278881791512529393768176295865058637443839948630435920364755158769558700401725729982303448806675404337152076182035887716084965628288039177878274454178560981629021091072141675384209759822095517042458068272797417658067509

## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [49]:
def encrypt(message, modulus, exp):
    # kod szyfrowania     
    
    # szyfrujemy litera po literze, zmieniając je na kod ascii
    
    message_encrypted = []
    for letter in message:
        letter = ord(letter)
        message_encrypted.append(int(pow(letter, exp, modulus)))
    return message_encrypted


message = "co tutaj sie dzieje"

message_encrypted = encrypt(message, public_key[0], public_key[1])
print(message_encrypted)



[...]


### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [51]:
def decrypt(message_encrypted, modulus, exp):
    message_decrypted = []
    for encrypted_letter in message_encrypted:
        message_decrypted.append(chr(pow(encrypted_letter, exp, modulus)))
        
    return ('').join(message_decrypted)


message_decrypted = decrypt(message_encrypted, *private_key)

print(message_decrypted)

co tutaj sie dzieje


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie 
Zademonstruj deformowalność podręcznikowego szyfrowania RSA